In [2]:
import django_init
import os, random, time
import time
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import pandas as pd
DF = pd.DataFrame
from django.conf import settings
from django_pandas.io import read_frame
from tqdm import tqdm
from main.models import *
from django_aux.models import *
from django_aux.utils import *
from django_aux.columns import *
from django_aux_timeperiods.models import *
from django_aux_geo.models import *
from request.models import Request
import calendar
from zoneinfo import ZoneInfo

In [3]:
SaleFactory.create_batch(size=10)

[<Sale: Dustin Anthony Westberry | 2022-08-08 07:31:39.130340+00:00 | utilities | 5349.18101914549>,
 <Sale: Jim Harry Ewing | 2021-12-22 16:41:40.875269+00:00 | resturant | 85.2442988455418>,
 <Sale: Megan Denise Bennett | 2022-04-17 09:01:53.623530+00:00 | entertainment | 3541.15196250641>,
 <Sale: Kevin Pablo Johnson | 2021-04-22 15:17:26.390675+00:00 | misc | 320.278935630356>,
 <Sale: Melissa Rhonda Marquez | 2022-05-29 21:28:52.433545+00:00 | utilities | 5464.63501849794>,
 <Sale: Carlton James Jung | 2021-04-18 10:37:03.851407+00:00 | resturant | 4338.26599714281>,
 <Sale: Joseph Ronald Barton | 2022-10-26 17:38:52.232553+00:00 | misc | 3079.52949078558>,
 <Sale: Brittany Kathleen Beuth | 2021-02-28 20:52:08.422947+00:00 | entertainment | 1435.99802018656>,
 <Sale: Jim Harry Ewing | 2023-07-25 19:01:14.398702+00:00 | utilities | 1609.87490171534>,
 <Sale: Peggy Nancy Keesling | 2023-06-06 21:52:54.967667+00:00 | groceries | 2879.60740024173>]

In [2]:
qs = Sale.objects.values()
create_view_from_qs(qs, view_name='sales', materialized=False, read_only_users=['user_readonly'])
create_view_from_qs(qs, view_name='sales_mat', ufields=['id'], read_only_users=['user_readonly'])

In [6]:
class FakeRequest:
    def __init__(self, GET=None, POST=None, META=None):
        self.GET = GET if GET else {}
        self.POST = POST if POST else {}
        self.META = META if META else {}


from django_aux.views import InlineFormsetMixin

ifm = InlineFormsetMixin()
ifm.request = FakeRequest(GET = {'extra':'2'})

assert ifm.get_current_extra() == 2

In [33]:
freq = 'Y'
#= dt.datetime.now()
def get_target_period(n, freq):
    now = dt.datetime.now()
    if n == 0:
        return pd.Period(now, freq=freq)
    if n < 0:
        pr = pd.period_range(end = now, periods=abs(n)+1, freq=freq)
        return pr[0]
    else:
        pr = pd.period_range(start = now, periods=abs(n)+1, freq=freq)
        return pr[-1]


    # future = pd.period_range(now, periods=20, freq=freq)
    # pr2 = pd.period_range(end = now, periods=20, freq=freq)
get_target_period(1, freq='M')


t1 = time.time()
get_target_period(n=1_000_000, freq='Y')
t2 = time.time()

print(t2-t1)

0.001998424530029297


In [3]:
o = {i:i for i in range(10)}
print(type(o), o)

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}

In [6]:
o = [i for i in range(10)]
print(type(o), o)

<class 'list'> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [14]:
from bs4 import BeautifulSoup
import requests

def get_category(alpha2):
    html = requests.get(f'https://en.wikipedia.org/wiki/ISO_3166-2:{alpha2}').text
    soup = BeautifulSoup(html, 'html.parser')
    try:
        cat = soup.get_text().split('are defined for')[1].split('.')[0].strip().split(' ')[-1]
    except:
        return None
    rtups = (
        ('ies','y'), ('provinces','province'), ('districts','district')
    )
    for rtup in rtups:
        cat = cat.replace(*rtup)
    return cat

get_category('LR')

'county'

In [37]:
def get_subdiv_df(country):
    alpha2 = country.alpha2
    url = f'https://en.wikipedia.org/wiki/ISO_3166-2:{alpha2}'
    try:
        dfs = pd.read_html(url)
    except:
        return DF()
    tdf = DF()
    for ttdf in dfs:
        if 'Code' in ttdf.columns: 
            tdf = ttdf
            break 
    if tdf.empty:
        return DF()
        
    df = tdf.copy().iloc[:,:2]
    df.columns = ['code','name']
    try:
        df['category'] = tdf['Subdivision category']
        df['category'] = df.category.str.lower()
    except:
        pass
    if 'category' not in df.columns:
        category = get_category(alpha2)
        df['category'] = category
    df['alpha2'] = country.alpha2
    return df
        
df = DF()
country = Country.objects.order_by('?').first()
for country in tqdm(Country.objects.order_by('?')):
    tdf = get_subdiv_df(country)
    if tdf.empty:
        continue
    df = pd.concat([df, tdf])

100%|██████████| 249/249 [02:46<00:00,  1.50it/s]


In [36]:
df

,code,name,category
0,SR-BR,Brokopondo,districts
1,SR-CM,Commewijne,districts
2,SR-CR,Coronie,districts
3,SR-MA,Marowijne,districts
4,SR-NI,Nickerie,districts
...,...,...,...
2,ST-03,Caué,district
3,ST-04,Lembá,district
4,ST-05,Lobata,district
5,ST-06,Mé-Zóchi,district


In [17]:
cats = []
for country in tqdm(Country.objects.exclude(alpha2=None)):
    cat = get_category(country.alpha2)
    if cat not in cats:
        cats.append(cat)


100%|██████████| 248/248 [01:43<00:00,  2.39it/s]


In [18]:
cats

['province',
 None,
 'county',
 'parishes',
 'dependency',
 'regions',
 'territory',
 'states',
 '(i',
 'districts',
 'governorates',
 'hyphen',
 'city',
 'departments',
 'municipality',
 'status',
 'district',
 'prefectures',
 '(mintaqah[1])',
 '(PRC)',
 'islands',
 'Eswatini',
 'Åland)',
 'divisions',
 'units',
 'rights',
 'region',
 'Ireland',
 'islands)',
 'popularates',
 'communes',
 'cantons',
 'atolls',
 'councils',
 'capital',
 'unit',
 'quarters',
 'Netherlands',
 'areas',
 'voivodeships',
 'entity',
 'administration',
 'ward',
 'Kampala)',
 '2014',
 'emirates',
 'islands"',
 'republic',
 'precincts']

In [32]:
format = {
    'AD': {'col_names':['code', 'name'], 'category':'parish'},
    'AE': {'col_names':['code', '_', 'name', '_'], 'category':'emirate'},
    'AF': {'col_names':['code', '_', 'name', '_'], 'category':'province'},
    'AG': {'col_names':['code', 'name', 'category']},
    'AL': {'col_names':['code', 'name'], 'category': 'county'},
    'AM': {'col_names':['code', '_' ,'name', '_', 'category']},
}
d = format[alpha2]
df.columns = d['col_names']
if d.get('category'):
    df['category'] = d.get('category')
df

,code,_,name,_,category
0,AM-ER,Erevan,Yerevan,Երևան,city
1,AM-AG,Aragac̣otn,Aragatsotn,Արագածոտն,region
2,AM-AR,Ararat,Ararat,Արարատ,region
3,AM-AV,Armavir,Armavir,Արմավիր,region
4,AM-GR,Geġark'unik',Gegharkunik,Գեղարքունիք,region
5,AM-KT,Kotayk',Kotayk,Կոտայք,region
6,AM-LO,Loṙi,Lori,Լոռի,region
7,AM-SH,Širak,Shirak,Շիրակ,region
8,AM-SU,Syunik',Syunik,Սյունիք,region
9,AM-TV,Tavuš,Tavush,Տավուշ,region


In [5]:
Country.create_objs()

In [6]:
country = Country.objects.order_by('?').first()
print(country, country.alpha2)
SubDivision.create_objs_from_country(country)
country.subdivision_set.all()



GEO GE


ValueError: Length mismatch: Expected axis has 5 elements, new values have 3 elements

In [7]:
country.alpha2

'MA'

In [28]:
tzinfo = timezone.now().tzinfo
n = 0

now = dt.datetime.now()
if tzinfo:
    now = now.astimezone(tzinfo)

cperiod = pd.Period(now, freq='M')
if n==0:
    cperiod
elif n < 0:
    kwargs = {'end':cperiod}
    i = 0
elif n > 0:
    kwargs = {'start':cperiod}
    i = -1

pd.period_range(**kwargs, periods=abs(n))[i]

IndexError: index -1 is out of bounds for axis 0 with size 0

In [4]:
dtg = timezone.now() + dt.timedelta(53)
for model in [Year, Month, Week, Day]:
    obj,_ = model.get_or_create_from_date(dtg.date())
    print(obj, obj.period, obj.get_rel_status())


2022 2022 present
2022-11 2022-11 future
2022 W45 2022-11-07/2022-11-13 future
2022-11-13 2022-11-13 future


datetime.timezone(datetime.timedelta(days=-1, seconds=72000), 'Eastern Daylight Time')

In [ ]:
week,c = Week.objects.get_or_create(year_num=2022, week_num=50)
week.period

In [ ]:
year_num=2022
for year_num in range(2000,2020):
    start = pd.to_datetime(dt.date(year_num,1,1)).to_period('W').start_time
    end = pd.to_datetime(dt.date(year_num,12,31)).to_period('W').end_time
    dates = pd.date_range(start, end, freq='W')
    if start.week != 1:
        dates = dates[1:]
    if end.week < 52:
         dates = dates[:-1]

pd.Period(dates[0], freq='W')


In [ ]:
pd.to_datetime(dt.date(2022,1,2)).to_period('W').week

In [ ]:
pd.date_range(start=)

In [ ]:
tuple(zip([calendar.month_name[i] for i in range(1,13)],[calendar.month_name[i] for i in range(1,13)]))
tuple(zip([calendar.month_abbr[i] for i in range(1,13)],[calendar.month_abbr[i] for i in range(1,13)]))

cal = calendar.Calendar()


In [ ]:
person = Person.objects.get(id=3)

read_frame(person.history.all())

In [ ]:
len('october')